In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
list_file = 'experiments_list.txt'
with open(list_file, 'r') as f:
    dirs = [f'experiments/{line.strip()}' for line in f if line.strip() and not line.strip().startswith('#')]

In [3]:
# Load logs.csv from each directory into a dictionary of DataFrames
dfs = {}
for d in dirs:
    csv_path = os.path.join(d, 'logs.csv')
    if os.path.isfile(csv_path):
        dfs[d] = pd.read_csv(csv_path)
    else:
        print(f'Warning: {csv_path} not found, skipping.')

# Optionally, concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(
    dfs.values(), 
    keys=dfs.keys(), 
    names=['experiment', 'row']
).reset_index(level='row', drop=True).reset_index().rename(columns={'index': 'experiment'})

# Display the combined DataFrame
df = combined_df.query('itr>0')[['experiment','iteration_time']].copy().groupby('experiment').mean().reset_index()

In [4]:
parts = df.experiment.str.split('/', expand=True)
df['Experiment'] = parts[1]
df['Method'] = parts[2]
df['Loss'] = parts[3]

In [5]:
df['Loss'].replace(
    {'SOCM_adjoint': 'SOCM (adjoint)',
     'log_variance': 'Log variance',
     'rel_entropy': 'Relative entropy',
     'pinn_GAUSS': 'PINN',
     'rel_GAUSS': 'Relative loss',
     'ritz_GAUSS': 'Deep ritz loss',
     'var_GAUSS': 'Variational loss'},
     inplace=True
)

df['Experiment'].replace(
    {'OU_anisotropic_d20': 'Stable OU (ANISO)',
     'OU_stable_d20': 'Stable OU (ISO)',
     'OU_hard_d20': 'Unstable OU',
     'double_well_d10': 'Double Well'},
     inplace=True
)

/tmp/ipykernel_897776/3065527133.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Loss'].replace(
/tmp/ipykernel_897776/3065527133.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: v

In [7]:
pivot = df.pivot_table(
    index=['Method', 'Loss'],
    columns='Experiment',
    values='iteration_time',
    aggfunc='mean'
)

In [10]:
display(pivot)

Experiment                 Double Well  Stable OU (ANISO)  Stable OU (ISO)  \
Method   Loss                                                                
COMBINED Log variance         0.744688           0.568906         0.458537   
         Relative entropy     0.815773           0.613510         5.626219   
         SOCM                 1.110253           0.738791         0.603972   
         SOCM (adjoint)       0.739245           0.569857         0.466268   
EIGF     Deep ritz loss       1.285113           0.363406         0.194309   
         PINN                 1.671822           0.585661         0.403245   
         Relative loss        1.672093           0.786410         0.403876   
         Variational loss     1.276355           0.363305         0.198153   
IDO      Log variance         0.211903           0.232888         0.207384   
         Relative entropy     0.454642           7.750473         0.398909   
         SOCM                 1.365313           1.152043         0.807243   
         SOCM (adjoint)       0.232870           0.248138         0.223761   

Experiment                 Unstable OU  
Method   Loss                           
COMBINED Log variance         0.805939  
         Relative entropy     0.920086  
         SOCM                 1.196630  
         SOCM (adjoint)       0.808442  
EIGF     Deep ritz loss       0.194784  
         PINN                 0.405677  
         Relative loss        0.404723  
         Variational loss     0.198188  
IDO      Log variance         2.638715  
         Relative entropy     7.000916  
         SOCM                 4.789852  
         SOCM (adjoint)       2.613046

In [11]:
latex_str = pivot.to_latex(
    caption='Iteration times by experiment, method and loss',
    label='tab:iteration_times',
    escape=True,            # <-- ensure special chars (_ & %) get backslash-escaped
    multicolumn=True,
    multicolumn_format='c',
    float_format="%.3f",
    bold_rows=True
)
print(latex_str)

\begin{table}
\caption{Iteration times by experiment, method and loss}
\label{tab:iteration_times}
\begin{tabular}{llrrrr}
\toprule
 & Experiment & Double Well & Stable OU (ANISO) & Stable OU (ISO) & Unstable OU \\
Method & Loss &  &  &  &  \\
\midrule
\multirow[t]{4}{*}{\textbf{COMBINED}} & \textbf{Log variance} & 0.745 & 0.569 & 0.459 & 0.806 \\
\textbf{} & \textbf{Relative entropy} & 0.816 & 0.614 & 5.626 & 0.920 \\
\textbf{} & \textbf{SOCM} & 1.110 & 0.739 & 0.604 & 1.197 \\
\textbf{} & \textbf{SOCM (adjoint)} & 0.739 & 0.570 & 0.466 & 0.808 \\
\cline{1-6}
\multirow[t]{4}{*}{\textbf{EIGF}} & \textbf{Deep ritz loss} & 1.285 & 0.363 & 0.194 & 0.195 \\
\textbf{} & \textbf{PINN} & 1.672 & 0.586 & 0.403 & 0.406 \\
\textbf{} & \textbf{Relative loss} & 1.672 & 0.786 & 0.404 & 0.405 \\
\textbf{} & \textbf{Variational loss} & 1.276 & 0.363 & 0.198 & 0.198 \\
\cline{1-6}
\multirow[t]{4}{*}{\textbf{IDO}} & \textbf{Log variance} & 0.212 & 0.233 & 0.207 & 2.639 \\
\textbf{} & \textbf{Relative e